In [4]:
# Cargamos las librerías necesarias
# Load the necessary libraries

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import os
import pickle
import re

import time
import json
from jsonschema import validate, ValidationError

import openai
import spacy
from docx import Document

from anthropic import Anthropic
anthropic = Anthropic(api_key='')  # Replace with your Anthropic API key
openai.api_key = ''  # Replace with your OpenAI API key

In [5]:
# Debes introducir la ruta de la carpeta donde los archivos intermedios que permiten verificar el razonamiento del sistema van a ser almacenados
# You must enter the path of the folder where the intermediate files that allow verifying the system's reasoning will be stored

folder_path = r'your_path'

In [6]:
query = input("Introduce los hechos del caso: ")

### Funciones de Limpieza y Procesamiento / Cleaning and Processing Functions

En esta sección se presentan las funciones básicas que utiliza nuestra Intelgencia Artificial para el procesamiento de la data textual

---

In this section, the basic functions used by our Artificial Intelligence for processing textual data are presented.

In [7]:
# Para limpieza y procesamiento de texto
# For text cleaning and processing


def clean_text_column(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    df[column_name] = df[column_name].str.replace(r'[\n\r\t]', ' ', regex=True).str.strip()
    df[column_name] = df[column_name].str.replace(r'\s+', ' ', regex=True)
    df[column_name] = df[column_name].str.replace(r'\xa0', ' ', regex=True)
    df[column_name] = df[column_name].str.replace('*', '', regex=False)
    df[column_name] = df[column_name].astype(str)
    df[column_name] = df[column_name].str.replace(r'[\uF02D\uF02E]', '', regex=True) 
    df[column_name] = df[column_name].str.lower()
    
    return df

In [8]:
# Para generar resúmenes de texto
# For generating text summaries

def generate_summaries(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Elabora un resumen detallado en formato de párrafos del siguiente extracto, indicando en la última oración las páginas resumidas"},
            
            {"role": "user", "content": prompt}
        ],
        max_tokens=16383,
        temperature=0
    )
    return response.choices[0].message['content']

In [9]:
# Para la lectura y procesamiento de json schemas
# For reading and processing JSON schemas

SCHEMA_DIR = r'your_schemas_path'     

def load_json_schema(schema_name):
    schema_path = os.path.join(SCHEMA_DIR, schema_name)
    with open(schema_path, 'r') as file:
        return json.load(file)

schema = load_json_schema("schema.json")

def extract_json_from_response(response_content):
    match = re.search(r"\{.*\}", response_content, re.DOTALL)
    if match:
        return match.group(0)
    return None

def convert_to_string(data):
    for key in data:
        if isinstance(data[key], list):
            data[key] = [str(item) if item is not None else "N/A" for item in data[key]]
    return data

In [18]:
# Download the model if not installed
# Descarga el modelo si no ha sido instalado

try:
    spacy.load("es_core_news_sm")
    print("Model 'es_core_news_sm' is already installed.")
except OSError:
    # Download the model if not installed
    print("Downloading 'es_core_news_sm'...")
    os.system("python -m spacy download es_core_news_sm")
    print("Model installed successfully!")

Model installed successfully!


In [10]:
# Para la generación de embeddings 
# For generating embeddings

def get_embeddings(texts):
    max_tokens = 8191  
    rate_limit_wait_time = 60  
    timeout_wait_time = 20 

    def split_texts(texts, max_tokens):
        """Split texts into chunks based on the maximum token limit."""
        chunks = []
        current_chunk = []
        current_length = 0
        for text in texts:
            text_length = len(text.split())  
            if current_length + text_length > max_tokens:
                chunks.append(current_chunk)
                current_chunk = [text]
                current_length = text_length
            else:
                current_chunk.append(text)
                current_length += text_length
        if current_chunk:
            chunks.append(current_chunk)
        return chunks

    def get_embeddings_for_chunk(chunk):
        """Get embeddings for a single chunk of texts."""
        while True:
            try:
                if not all(isinstance(c, str) for c in chunk):
                    raise ValueError("Chunk contains non-string elements.")
                
                response = openai.Embedding.create(
                    model="text-embedding-3-small", 
                    input=chunk 
                )
                return [data['embedding'] for data in response['data']]
            except openai.error.InvalidRequestError as e:
                print(f"Invalid request: {e}. Chunk: {chunk}")
                raise
            except openai.error.RateLimitError:
                print(f"Rate limit exceeded. Waiting for {rate_limit_wait_time} seconds...")
                time.sleep(rate_limit_wait_time)
            except openai.error.APIConnectionError:
                print(f"Connection timed out. Waiting for {timeout_wait_time} seconds...")
                time.sleep(timeout_wait_time)
            except Exception as e:
                print(f"An error occurred: {e}")
                raise

    chunks = split_texts(texts, max_tokens)
    embeddings = []
    for i, chunk in enumerate(chunks):
        try:
            embeddings.extend(get_embeddings_for_chunk(chunk))
        except Exception as e:
            print(f"Error processing chunk {i}: {chunk}") 
            continue

    return np.array(embeddings)

### Preparando la base de datos que contiene las decisiones en materia de Libre Competencia / Preparing the database containing decisions on Competition Law

Se cargan las bases de datos que servirán como fuente de conocimiento externo para nuestro sistema.

---

The databases that will serve as external knowledge sources for our system are loaded.

In [11]:
# Cargamos y procesamos las bases de datos necesarias
# Load and process the necessary databases

jurisprudencia = pd.read_pickle(r"cl_corpus_2009_2024.pkl")
metadata = pd.read_pickle(r"metadata.pkl") 
jurisprudencia = pd.merge(jurisprudencia, metadata, on=['ID', 'Subfolder'])
url_df = pd.read_csv(r"pdf_urls.csv")
url_df['ID'] = url_df['ID'].str.replace('.pdf', '')
jurisprudencia = pd.merge(jurisprudencia, url_df, how='left', on='ID')
replace_dict = {
    'ccp_2009_2024_ant': 'Cuerpo Colegiado Permanente del OSIPTEL',
    'tsc_2009_2024_ant': 'Tribunal de Solución de Controversias del OSIPTEL',
    'sdc_2009_2024_ant': 'Sala de Defensa de la Competencia del INDECOPI',
    'clc_2009_2024_ant': 'Comisión de Defensa de la Libre Competencia del INDECOPI'
}

jurisprudencia['Subfolder'] = jurisprudencia['Subfolder'].replace(replace_dict)

dl_1034 = pd.read_excel(r"dl_1034.xlsx") # Contiene los artículos del TUO de la LRCA, así como los objetivos a interpretar por artículo

jurisprudencia = clean_text_column(jurisprudencia, 'Text')
dl_1034 = clean_text_column(dl_1034, 'rule')

In [12]:
# Complementamos la carga de la base de datos con los embeddings
# Complete the database loading with the embeddings

embeddings_path = r"embeddings.pkl"

with open(embeddings_path, 'rb') as file:
    data = pickle.load(file)
    summary_embeddings = data['summary_embeddings']
    saved_metadata = data['metadata']
    saved_summaries = data['summaries']

df_to_merge = jurisprudencia[['Resolución', 'Subfolder', 'Fecha', 'Expediente', 'ID', 'url']].drop_duplicates(subset='ID')
saved_metadata = pd.merge(saved_metadata, df_to_merge, on='ID', how='left')

with open(r"tfidf_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

with open(r"tfidf_matrix_cl_corpus.pkl", "rb") as f:
    tfidf_matrix = pickle.load(f)

d:\Programas\anaconda3\envs\ulises\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Programas\anaconda3\envs\ulises\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### Módulo Issue / Issue Module

El objetivo principal de este módulo es formular el **Legal Judgment Prediction (LJP)** propuesto, en su forma clásica. Este enfoque se centra en determinar si los hechos presentados en un caso constituyen o no una violación a la ley. 

En este contexto particular, el **issue** planteado busca responder si los hechos específicos del caso infringen los requisitos establecidos en la **ley de competencia** para sancionar una práctica colusoria horizontal.

En este módulo, utilizamos exclusivamente el siguiente código para plantear el problema legal a resolver.


------


In this module, we exclusively use the following code to frame the legal question to be addressed:


The primary goal of this module is to implement the proposed **Legal Judgment Prediction (LJP)** in its classical form. This approach focuses on determining whether the facts presented in a case constitute a violation of the law. 

In this specific context, the **issue** aims to determine whether the case's specific facts infringe the requirements set by **competition law** to sanction a horizontal collusive practice.

In this module, we exclusively use the following code to frame the legal question to be addressed:


In [13]:
issue = f"¿Los siguientes hechos '{query}' SI cumple O NO cumple con los requisitos que establece la ley de competencia para sancionar una conducta?"

### Módulo Regla / Rule Module

Este módulo consta de una secuencia de pasos diseñados para facilitar la aplicación e interpretación de los artículos legales relevantes para resolver prácticas colusorias horizontales.

1. **Generación de Preguntas Específicas**  
   Se utiliza una función para generar preguntas precisas sobre la aplicación de los artículos relevantes al caso. Esta función toma como datos de entrada las columnas **goal**, **rule** y los **hechos del caso**, para formular las preguntas más adecuadas que permitan interpretar el artículo y su aplicación a los hechos.

2. **Recuperación de Decisiones Relevantes**  
   Las preguntas generadas por un modelo de lenguaje son utilizadas para recuperar decisiones del INDECOPI u OSIPTEL. Este proceso de recuperación emplea dos algoritmos:  
   - **Embedding Semántico**: Utiliza el modelo `text-embedding-3-small` para capturar relaciones semánticas entre la Pregunta Específica y los extractos de las decisiones.
   - **Análisis TF-IDF**: Complementa la búsqueda con un enfoque estadístico basado en frecuencia de términos.

   Ambos algoritmos identifican los extractos más relevantes que puedan contener las respuestas a las preguntas. Estos extractos se combinan y se recupera la página que contiene el extracto, junto con dos páginas consecutivas para garantizar que se aborde la mayor cantidad de contexto posible.

3. **Resumir la Información Extraída**  
   El contenido extraído se resume utilizando otro modelo de lenguaje, condensando las páginas recuperadas en información relevante y concisa.

4. **Responder las Preguntas Específicas**  
   Finalmente, un modelo de lenguaje utiliza los resúmenes generados para responder a las preguntas específicas sobre la aplicación de los artículos relevantes para evaluar la legalidad de una práctica colusoria horizontal.

Este proceso asegura un análisis robusto, integrando la recuperación de información generada por las agencias de competencia, la síntesis de información y respuestas precisas a preguntas específicas en Derecho de la Libre Competencia.

---

This module consists of a sequence of steps designed to facilitate the application and interpretation of relevant legal provisions for resolving horizontal collusive practices.

1. **Generating Specific Questions**  
   A function is used to generate precise questions about the application of relevant articles to the case. This function takes input from the **goal**, **rule**, and **facts of the case** columns to formulate the most appropriate questions for interpreting the article and its application to the facts.

2. **Retrieving Relevant Decisions**  
   The questions generated by a language model are used to retrieve decisions from **INDECOPI** or **OSIPTEL**. This retrieval process employs two algorithms:  
   - **Semantic Embedding**: Utilizes the `text-embedding-3-small` model to capture semantic relationships between the Specific Question and decision excerpts.  
   - **TF-IDF Analysis**: Complements the search with a statistical term-frequency approach.  

   Both algorithms identify the most relevant excerpts that may contain answers to the questions. These excerpts are combined, retrieving the page containing the excerpt along with two consecutive pages to ensure adequate context is captured.

3. **Summarizing Extracted Information**  
   The extracted content is summarized using another language model, condensing the retrieved pages into concise and relevant information.

4. **Answering Specific Questions**  
   Finally, a language model uses the generated summaries to answer the specific questions regarding the application of the relevant articles to assess the legality of a horizontal collusive practice.

This process ensures a robust analysis, integrating the retrieval of information generated by competition agencies, the synthesis of information, and precise answers to specific questions in Competition Law.


In [14]:
# Vamos a elaborar la data de entrada que servirá para obtener nuestras preguntas sobre la correcta aplicación del artículo de la LRCA
# Es importante que las preguntas, aunque abstractas, sean capaces de mantener cierta orientación hacia los Hechos del Caso. 
# Es por tal razón que se incluye el Objetivo Planteado y los Hechos del Caso, así como el artículo que debe ser interpretado.

# We will create the input data that will serve to generate our questions about the correct application of the LRCA article
# It is important that the questions, although abstract, are able to maintain some orientation towards the Case Facts.
# For this reason, the Stated Objective and the Case Facts are included, as well as the article that must be interpreted.


dl_1034['input'] = dl_1034.apply(lambda row: f'Objetivo Planteado:{row["goal"]} Hechos del Caso {query} Artículo a Interpretar: {row["rule"]}', axis=1)

In [15]:
def issue_per_rule(text, max_tokens=200, temperature=0, top_p=1):
    """
    Genera preguntas abstractas basadas en un texto de caso usando la API de Claude de Anthropic.
    
    La función toma un texto que contiene el artículo que debe ser aplicado a los hechos de un caso, 
    así como directivas para mejorar las preguntas mediante objetivos, y lo procesa a través 
    del modelo de lenguaje Claude y genera dos preguntas abstractas que ayudan a analizar el caso. 
    Las preguntas se generan de forma genérica, evitando nombres específicos y usando el término 
    "empresa" para referirse a las entidades involucradas. Es especialmente útil para análisis 
    de casos legales o empresariales donde se necesita abstraer los detalles específicos para 
    enfocarse en los principios o problemas fundamentales del caso.
    """
    try:
        message = anthropic.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            system=
                "Elabora dos preguntas abstractas que sirvan para cumplir con el Objetivo Planteado a partir de los Hechos del Caso. No menciones nombres, solo refierete a ellos como empresa "
                "Responde en el siguiente formato:"
                "¿(Pregunta Abstracta 1)?"
                "¿(Pregunta Abstracta 2)?",
            messages=[{
                "role": "user",
                "content": text
            }]
        )
        return message.content[0].text if isinstance(message.content, list) else message.content

    except Exception as e:
        print(f"Error occurred: {e}")
        return None

dl_1034['issue_per_rule'] = dl_1034['input'].apply(issue_per_rule)

In [16]:
# Recuperación mediante embeddings semánticos
# Retrieval using semantic embeddings


embeddings_rows = [] 

# Iterar sobre cada fila del dataFrame 'dl_1034'
# Iterate over each row of the 'dl_1034' DataFrame

for i, row in dl_1034.iterrows():
    input_text = row['issue_per_rule']  
    
    query_embedding = get_embeddings([input_text])[0]
    
    cos_similarities = cosine_similarity([query_embedding], summary_embeddings)[0]
    
    # Obtener los índices de las 4 filas más similares (ordenadas de mayor a menor)
    # Get the indices of the 4 most similar rows (sorted from highest to lowest)

    top_indices = cos_similarities.argsort()[-4:][::-1]
    
    for idx in top_indices:
        # Acceder a los metadatos de la fila similar
        # Access the metadata of the similar row

        similar_row = saved_metadata.iloc[idx] 
        
        # Agregar a la lista información de la fila original, la similitud y los metadatos de la fila similar
        # Add to the list information from the original row, the similarity, and the metadata of the similar row

        embeddings_rows.append({
            "section": row['section'],               
            "rule": row['rule'],                    
            "questions": row['issue_per_rule'],      
            "Similarity": cos_similarities[idx],    
            "ID": similar_row['ID'],                 
            "age": similar_row['Page'],            
            "authority": similar_row['Subfolder'],   
            "decision": similar_row['Resolución'], 
            "case_file": similar_row['Expediente'], 
            "date": similar_row['Fecha'],           
            "link": similar_row['url'],            
        })

embeddings_df = pd.DataFrame(embeddings_rows)

In [19]:
# Recuperación mediante TF-IDF
# Retrieval using TF-IDF

nlp = spacy.load("es_core_news_sm")

# Función para preprocesar texto: lematización y eliminación de stopwords y puntuación
# Function to preprocess text: lemmatization and removal of stopwords and punctuation

def preprocess_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Aplicar preprocesamiento a la columna 'issue_per_rule'
# Apply preprocessing to the 'issue_per_rule' column

dl_1034['Processed_Text'] = dl_1034['issue_per_rule'].apply(preprocess_text)

ti_rows = []  

# Iterar sobre cada fila del DataFrame 'dl_1034'
# Iterate over each row of the 'dl_1034' DataFrame

for i, row in dl_1034.iterrows():
    query_text = row['Processed_Text']  
    
    # Transformar el texto de consulta al mismo espacio vectorial que 'tfidf_matrix'
    # Transform the query text into the same vector space as 'tfidf_matrix'

    query_vector = vectorizer.transform([query_text])
    
    # Calcular la similitud coseno entre el vector de consulta y toda la matriz tfidf
    # Calculate the cosine similarity between the query vector and the entire tfidf matrix

    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Obtener los índices de las 4 filas más similares (ordenadas de mayor a menor similitud)
    # Get the indices of the 4 most similar rows (sorted from highest to lowest similarity)

    top_indices = similarities.argsort()[-4:][::-1]
    
    # Para cada fila similar, almacenar la información relevante
    # For each similar row, store the relevant information

    for idx in top_indices:
        # Acceder a los metadatos de la fila similar
        # Access the metadata of the similar row

        similar_row = jurisprudencia.iloc[idx]  
        
        # Agregar a la lista la información de la fila original y la similar
        # Add the original row information and the similar row information to the list
        
        ti_rows.append({
            "section": row['section'],               
            "rule": row['rule'],                     
            "questions": row['issue_per_rule'],     
            "Similarity": similarities[idx],      
            "ID": similar_row['ID'],                 
            "Page": similar_row['Page'],            
            "authority": similar_row['Subfolder'],   
            "decision": similar_row['Resolución'],   
            "case_file": similar_row['Expediente'],  
            "date": similar_row['Fecha'],            
            "link": similar_row['url'],              
        })

ti_df = pd.DataFrame(ti_rows)
ti_df = ti_df.drop_duplicates(subset=['ID', 'Page'], keep='first')

In [20]:
# Combinar los DataFrames 'embeddings_df' y 'ti_df' en uno solo
# Combine the 'embeddings_df' and 'ti_df' DataFrames into one

retrieved_data = pd.concat([embeddings_df, ti_df], ignore_index=True)

# Crear una lista para almacenar las filas adicionales
# Create a list to store the additional rows

new_rows = []

# Iterar a través de cada fila en 'retrieved_data'
# Iterate through each row in 'retrieved_data'

for _, row in retrieved_data.iterrows():
    # Get the current ID and page of the row
    # Obtener el ID y la página actual de la fila
    current_id = row['ID']
    current_page = row['Page']
    
    # Filtrar el DataFrame 'jurisprudencia' por filas con el mismo ID
    # Filter the 'jurisprudencia' DataFrame for rows with the same ID

    df_filtered = jurisprudencia[jurisprudencia['ID'] == current_id]
    
    # Ordenar las filas filtradas por la columna 'Page'
    # Sort the filtered rows by the 'Page' column

    df_filtered = df_filtered.sort_values(by='Page')
    
    # Encontrar explícitamente la página actual y las siguientes dos páginas
    # Explicitly find the current page and the next two pages

    additional_rows = df_filtered[df_filtered['Page'] >= current_page].head(3)
    
    # Agregar las filas adicionales a la lista 'new_rows'
    # Add the additional rows to the 'new_rows' list

    for _, additional_row in additional_rows.iterrows():
        new_rows.append({
            # Incluir todas las columnas de 'retrieved_data'
            # Include all columns from 'retrieved_data'
            **row.to_dict(), 

            # Página adicional de 'jurisprudencia' 
            # Additional page from 'jurisprudencia'
            'pages': additional_row['Page'],

            # Texto adicional, si existe en 'jurisprudencia'   
            # Additional text, if it exists in 'jurisprudencia'
            'Text': additional_row.get('Text', None),  

        })

rule = pd.DataFrame(new_rows)

In [21]:
# Modificar la columna 'Text' para incluir el formato solicitado
# Modify the 'Text' column to include the requested format

rule['Text'] = rule.apply(lambda row: f'PÁGINA({row["pages"]}) {row["Text"]} PÁGINA({row["pages"]})', axis=1)

rule = (
    rule
    .groupby(['section','questions','rule', 'decision', 'link', 'authority', 'date', 'case_file'], as_index=False)
    .agg({
        'Text': lambda x: ' | '.join(x), 
    })
)

rule['summary'] = rule['Text'].apply(generate_summaries)

In [22]:
# Vamos a mantener este dataset que contiene los resúmenes de las tres páginas y los almacenaremos en un dataframe
# We will keep this dataset containing the summaries of the three pages and store it in a dataframe

rule_compendium = rule[['section','questions','rule', 'decision', 'link', 'authority', 'date', 'case_file', 'summary']].copy()

In [23]:
# Haremos expresa la metadata de las decisiones para que le sirva al modelo de lenguaje posteriormente
# We will make the metadata of the decisions explicit so it can be used by the language model later

rule['summary'] = rule.apply(lambda row: f'Resumen:{row["summary"]} El resumen pertenece a la Resolución {row["decision"]} emitida por la Autoridad:{row["authority"]}', axis=1)

rule = (
    rule
    .groupby(['section','questions','rule'], as_index=False)
    .agg({
        'summary': lambda x: ' | '.join(x),  # Combinar todos los resúmenes y tenerlos listos para responder a las preguntas
    })
)

In [24]:
# Vamos a generar el input para nuestro modelo de lenguaje
# We will generate the input for our language model


rule['input'] = rule.apply(lambda row: f'Jurisprudencia Relevante:\n{row["summary"]}\nPreguntas a Responder:\n{row["questions"]}', axis=1)

In [25]:
# Esta función va a responder a las preguntas específicas utilizando nuestra base de conocimiento de jurisprudencia.
# This function will answer specific questions using our jurisprudence knowledge base.

def answering_issue_per_rule(text, max_tokens=2013, temperature=0, top_p=1):
    try:
        message = anthropic.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            system=
                "Responde a las Preguntas planteadas a partir de la Jurisprudencia Relevante. Cita tus respuestas con (p.XX, Resolución XXX).",
            messages=[{
                "role": "user",
                "content": text
            }]
        )
        return message.content[0].text if isinstance(message.content, list) else message.content

    except Exception as e:
        print(f"Error occurred: {e}")
        return None

rule['answering_issue_per_rule'] = rule['input'].apply(answering_issue_per_rule)

In [26]:
# Almacenamos la información intermedia que sirve como fuente de verdad para la Inteligencia Artificial
# We store the intermediate information that serves as the truth source for the Artificial Intelligence

doc = Document()

doc.add_heading("Jurisprudencia Relevante", level=1)
for _, row in rule.iterrows():
    text = (
        f"Las siguientes preguntas: '{row['questions']}' sirven para interpretar el siguiente artículo: '{row['rule']}'\n\n"
        f"El siguiente texto contiene las respuestas a las preguntas: '{row['answering_issue_per_rule']}'"
        f"Los resumenes utilizados son los siguientes:\n\n{row['summary']}\n\n"
    )
    doc.add_paragraph(text)
output_path = (folder_path + '\\jurisprudencia_relevante.docx')
doc.save(output_path)

rule_compendium.to_excel(folder_path + '\\metadata_jurisprudencia.xlsx')

### Módulo de Aplicación / Application Module

Este módulo se centra en utilizar las respuestas a las preguntas específicas para interpretar un artículo de la ley de competencia en relación con los hechos del caso. El proceso incluye varios pasos estructurados para garantizar un análisis lógico y coherente:

1. **Interpretación Específica por Artículo**  
   Se emplea un modelo de lenguaje, contenido en la función `def application()`, instruido específicamente para razonar de manera lógica y coherente. Este modelo evalúa aspectos clave sobre la aplicación de un artículo específico de la ley a los hechos del caso.

2. **Agrupación de Respuestas por Área de Enfoque**  
   Después de obtener las respuestas para cada artículo aplicable, estas se agrupan en dos categorías:  
   - **Ámbito de Aplicación**: Respuestas que abordan la aplicabilidad de las normas referidas al ámbito de aplicación.  
   - **Interpretación de Prácticas Colusorias Horizontales**: Respuestas que analizan específicamente la aplicación de los artículos vinculados a las prácticas colusorias horizontales.  

   Luego, la misma función de aplicación se aplica a todos los informes relacionados.

3. **Consolidación de Respuestas**  
   Los dos grupos de respuestas se combinan en un objeto denominado `ira_text`. Este objeto contiene un resumen condensado pero completo sobre la aplicación de cada artículo a los hechos del caso. La consolidación garantiza que la información sea esencial y precisa, proporcionando una base sólida para responder al **issue**.

Al estructurar sistemáticamente la aplicación de cada artículo, este módulo ofrece un análisis claro, lógico y orientado al contexto para evaluar el caso bajo la ley de competencia.

---

This module focuses on utilizing the answers to specific questions to interpret an article of competition law in relation to the facts of the case. The process includes several structured steps to ensure logical and coherent analysis:

1. **Article-Specific Interpretation**  
   A language model, contained in the function `def application()`, is employed, specifically instructed to reason logically and coherently. This model evaluates key aspects regarding the application of a specific article of the law to the facts of the case.

2. **Grouping Responses by Focus Area**  
   After obtaining the answers for each applicable article, these are grouped into two categories:  
   - **Scope of Application**: Responses addressing the applicability of rules related to the scope of application.  
   - **Interpretation of Horizontal Collusive Practices**: Responses specifically analyzing the application of articles linked to horizontal collusive practices.  

   Then, the same application function is applied to all related reports.

3. **Consolidating Responses**  
   The two sets of responses are combined into an object called `ira_text`. This object contains a condensed yet complete summary of the application of each article to the facts of the case. The consolidation ensures that the information is essential and precise, providing a solid foundation to address the **issue**.

By systematically structuring the application of each article, this module delivers a clear, logical, and context-driven analysis to evaluate the case under competition law.


In [27]:
# Vamos a generar el input para nuestro modelo de lenguaje
# We will generate the input for our language model

rule['input_2'] = rule.apply(lambda row: f'Artículo a Aplicar:{row["rule"]}\nAyuda:\n{row["answering_issue_per_rule"]}\nHechos del Caso:\n{query}', axis=1)

In [28]:
def application(text, max_tokens=2013, temperature=0, top_p=1):
    """
    Función para generar una respuesta utilizando un modelo de lenguaje a gran escala.
    Este modelo analiza un texto proporcionado en base a un conjunto de instrucciones predefinidas 
    relacionadas con el análisis de un artículo legal.

    Parámetros:
        text (str): El texto de entrada para ser analizado.
        max_tokens (int): Número máximo de tokens que puede generar el modelo en la respuesta. 
                          Valor por defecto: 2013.
        temperature (float): Parámetro que controla la aleatoriedad de la respuesta. 
                             Valores bajos como 0 producen resultados más deterministas. Valor por defecto: 0.
        top_p (float): Parámetro que controla la probabilidad acumulada para filtrar las respuestas generadas. 
                       Valor por defecto: 1 (sin filtro).

    Retorna:
        str: El contenido generado por el modelo, dependiendo del análisis realizado.
        None: En caso de ocurrir una excepción, se devuelve `None` y se imprime el error.

    """
    try:
        message = anthropic.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            system=
                "Vamos a pensar paso."
                "1. ÁMBITO DE APLICACIÓN DEL ARTÍCULO:"
                "- Identifica primero a qué situaciones/casos aplica el artículo"
                "- Determina qué situaciones/casos están excluidos"
                "- Si el caso no está dentro del ámbito, detén el análisis aquí"
                "2. Solo si el caso está dentro del ámbito:"
                "- Extrae los demás criterios/requisitos específicos del Artículo"
                "- Identifica qué elementos deben probarse según el Artículo"
                "3. ANÁLISIS DE HECHOS:"
                "- Verifica si los hechos están dentro del ámbito de aplicación"
                "- Solo si están en el ámbito, analiza si cumplen los demás criterios"
                "4. CONCLUSIÓN:"
                "- Si los hechos no están en el ámbito, concluye que no aplica"
                "- Solo si están en el ámbito, analiza y concluye sobre los demás elementos",
            messages=[{
                "role": "user",
                "content": text
            }]
        )
        return message.content[0].text if isinstance(message.content, list) else message.content

    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [29]:
rule['application_1'] = rule['input_2'].apply(application)


In [30]:
ira = (
    rule
    .groupby(['section'], as_index=False)
    .agg({
        # Combinamos todas las respuestas por sección de análisis (ámbito de aplicación y acreditación de prácticas colusorias)
        # We combine all the answers by analysis section (scope of application and accreditation of collusive practices)
        
        'application_1': lambda x: ' | '.join(x), 
    })
)


In [31]:
ira['application_1'] = ira.apply(lambda row: f'Opiniones sobre los Artículos:{row["application_1"]}', axis=1)


In [32]:
ira_aa = ira[ira['section'] == 'aa']
ira_pc = ira[ira['section'] == 'pc']

In [33]:
def practicas_colusorias(text, max_tokens=2013, temperature=0, top_p=1):
    try:
        message = anthropic.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            system=
                "Vamos a pensar paso."
                "Determina si es una práctica colusoria a la que debería aplicarse la prohibición absoluta o relativa."
                "SI es inter marca Y accesoria a un acuerdo lícito Y tiene como objetivo explícito la fijación de precios o condiciones comerciales que inciden directamente en el precio, directamente en reparto de mercado,directamente en limitación de producción o reparto de contrataciones con el Estado, entonces es una prohibición absoluta."
                "SI uno de los requisitos antes mencionados no se satisface, entonces aplica la prohibición relativa y el balance de efectos correspondiente."
                "Debes dar una opinión sobre la existencia de una conducta anticompetitiva."
                "Responde en el siguiente formato:"
                "Análisis: (contraste de argumentos a favor de aplicar prohibición absoluta y argumentos a favor de aplicar prohibición relativa. Las prácticas sujetas a la prohibición absoluta no pueden ser exclusorias, es decir, si afecta la permanencia de competidores en el mercado y crea barreras a la entrada, es una prohibición relativa"
                "Respuesta: (no adelantar la respuesta hasta desarrollar el análisis)",
            messages=[{
                "role": "user",
                "content": text
            }]
        )
        return message.content[0].text if isinstance(message.content, list) else message.content

    except Exception as e:
        print(f"Error occurred: {e}")
        return None


In [34]:
ira_aa['application_2'] = ira_aa['application_1'].apply(application)
ira_pc['application_2'] = ira_pc['application_1'].apply(practicas_colusorias)

C:\Users\PC\AppData\Local\Temp\ipykernel_2800\1527804548.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ira_aa['application_2'] = ira_aa['application_1'].apply(application)
C:\Users\PC\AppData\Local\Temp\ipykernel_2800\1527804548.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ira_pc['application_2'] = ira_pc['application_1'].apply(practicas_colusorias)


In [35]:
ira =pd.concat([ira_aa,ira_pc])

In [36]:
ira_text = " ".join(ira['application_2'].astype(str))

In [37]:
def ira_resumen(text, max_tokens=2013, temperature=1, top_p=1):
    try:
        message = anthropic.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            system=
                "Sintetiza las conclusiones de la siguiente información",
            messages=[{
                "role": "user",
                "content": text
            }]
        )
        return message.content[0].text if isinstance(message.content, list) else message.content

    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [38]:
ira_text = ira_resumen(ira_text)

In [39]:
ira_text = f'¨Reglas-Aplicación: {ira_text} {issue}'

In [40]:
# Generamos los documentos revisados por la Inteligencia Artificial para posterior revisión
# We generate the documents reviewed by the Artificial Intelligence for later review

doc = Document()

doc.add_heading("Reglas y Aplicación", level=1)

for _, row in ira.iterrows():
    text = (
        f"Los siguientes textos pertenecen a la sección '{row['section']}'\n\n"
        f"El siguiente texto contiene la síntesis de las respuestas sobre la aplicación de los artículos:\n'{row['application_2']}'\n\n"
        f"Los siguientes textos contienen el análisis referido a la aplicación de un artículo concreto a los hechos del caso:\n\n{row['application_1']}\n\n"
    )
    doc.add_paragraph(text)
output_path = (folder_path + '\\reglas_aplicación.docx')
doc.save(output_path)

### Módulo de Conclusión / Conclusion Module

En este módulo, se utiliza una función que emplea un modelo de lenguaje con un prompt de Chain of Thought (CoT). La función recibe un texto como entrada y analiza si se cumplen los requisitos legales relacionados con el ámbito de aplicación (objetivo, subjetivo y territorial). Además, identifica si la práctica en cuestión es una prohibición absoluta o relativa, y si es relativa, evalúa si los efectos negativos son significativos y superan a los positivos, lo que justificaría que la práctica sea considerada prohibida.

La función devuelve un análisis detallado y una conclusión clara sobre si los hechos del caso constituyen una vulneración de la ley de competencia, considerando todos los aspectos relevantes.

---

In this module, a function is used that employs a language model with a Chain of Thought (CoT) prompt. The function takes a text input and analyzes whether the legal requirements related to the scope of application (objective, subjective, and territorial) are met. Additionally, it identifies whether the practice in question is an absolute or relative prohibition, and if it is relative, it assesses whether the negative effects are significant and outweigh the positive ones, which would justify considering the practice as prohibited.

The function returns a detailed analysis and a clear conclusion on whether the facts of the case constitute a violation of competition law, considering all relevant aspects.

In [41]:
def conclusion(text, max_tokens=2013, temperature=1, top_p=1):
    try:
        message = anthropic.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            system=
                "Vamos a pensar paso."
                "Vas a responder a la pregunta planteada a partir de la información proporcionada." 
                "Debes precisar si el ámbito de aplicación objetivo, subjetivo y territorial se cumplen. En caso no se cumplan, no se puede sancionar"
                "Debes identificar si se trata de una prohibición absoluta o relativa."
                "Si se trata de una prohibición relativa, debes analizar si los efectos negativos existen, son significativos y superan a los efectos positivos para que sea una práctica prohibida"
                "Responde en el siguiente formato"
                "Análisis"
                "Respuesta: (no debes adelantar tu respuesta, la respuesta debe ir como conclusión a tu análisis)",
            messages=[{
                "role": "user",
                "content": text
            }]
        )
        return message.content[0].text if isinstance(message.content, list) else message.content

    except Exception as e:
        print(f"Error occurred: {e}")
        return None

irac = conclusion(ira_text)

In [42]:
def conclusion_final(query, model):
    if not isinstance(query, str):
        raise ValueError("Input must be a string.")

    # Prompt
    prompt = (
        "Extrae y organiza los datos del esquema JSON sobre la investigación anticompetitiva. "
        "En 'anticompetitive', indica con un valor booleano (true o false) si hubo una vulneración a la ley de competencia. "
        "En 'grounds', proporciona una explicación detallada de las razones por las cuales hubo o no una vulneración a la ley de competencia. "
        "Devuelve exclusivamente en formato JSON, sin explicaciones ni comentarios, con el siguiente esquema:\n"
        "{\n"
        "  \"anticompetitive\": true o false,\n"
        "  \"grounds\": [\n"
        "    \"Explicación detallada de las razones\"\n"
        "  ]\n"
        "}"
    )
    # API request
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "user", "content": prompt + "\n\n" + query}],
        temperature=0.95,
        max_tokens=1500,
    )

    # Extract JSON from response
    response_content = response.choices[0].message['content']
    print("Raw API response content:", response_content)  # Debugging

    # Improved JSON extraction
    
    json_content = extract_json_from_response(response_content)
    if not json_content:
        print("No valid JSON found in response.")
        return None

    # Parse and validate the JSON
    try:
        data = json.loads(json_content)  # Parse JSON
        validate(instance=data, schema=schema['schema'])  # Validate with updated schema
        return data
    except ValidationError as ve:
        print("Validation error:", ve.message)
        print("Offending data:", json_content)
        return None
    except json.JSONDecodeError as je:
        print("JSON parsing error:", je)
        print("Offending response content:", response_content)
        return None
    
if __name__ == "__main__":
    model = "gpt-4o"  # Change this to the desired model

    # Use the joined_text as the query input
    try:
        result = conclusion_final(irac, model)
        if result:
            print("Extracted Valid Data")
        else:
            print("No valid data extracted.")
    except ValueError as ve:
        print("Input error:", ve)

Raw API response content: ```json
{
  "anticompetitive": false,
  "grounds": [
    "No cumple con los requisitos para ser sancionada porque, aunque es una práctica colusoria horizontal intramarca sujeta a prohibición relativa, no se demuestran efectos negativos significativos en el mercado.",
    "La duración de los efectos fue corta, solo 3 meses, y no afectó el volumen de servicios.",
    "Los precios acordados se mantuvieron dentro del rango de mercado, y existían abundantes alternativas competitivas, ya que los talleres multimarca representaban el 60% del mercado.",
    "No se evidencian efectos negativos significativos que superen los beneficios."
  ]
}
```
Extracted Valid Data


In [43]:
prediction_final = pd.DataFrame(result)
prediction_final = (
    prediction_final
    .groupby(['anticompetitive'], as_index=False)
    .agg({
        'grounds': lambda x: ''.join(x),
    })
)
prediction_final.to_excel(folder_path + '\\predicción_ljp.xlsx')

In [44]:
# Genera el Informe IRAC

doc = Document()

doc.add_heading("Informe IRAC", level=1)

text = (
    f"Issue:{issue}\n"
    f"RAC:\n'{irac}'\n"
)
doc.add_paragraph(text)

output_path = (folder_path + '\\irac.docx')
doc.save(output_path)